# SBA National Loan Analysis

# Modeling - Part 3 - XGBoosting Classifier

## 1. Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, f1_score, matthews_corrcoef, confusion_matrix

from library.preprocessing import processing_pipeline
from library.modeling import (createModel, createClassificationMetrics,
                             runGridSearchAnalysis, createConfusionMatrix, createFeatureImportanceChart)

## 2. Previewing Data

In [2]:
sba_loans = pd.read_csv('./../data/processed/sba_national_processed_final.csv')

pd.set_option('display.max_columns', None)

In [3]:
sba_loans.head()

,Term,NoEmp,CreateJob,RetainedJob,DisbursementGross,GrAppv,SBA_Appv,NAICS_sectors,unemployment_rate,gdp_growth,gdp_annual_change,inflation_rate,inf_rate_annual_chg,NewExist_existing_business,NewExist_new_business,UrbanRural_rural,UrbanRural_urban,isFranchise_not_franchise,RevLineCr_v2_N,RevLineCr_v2_Y,LowDoc_v2_N,LowDoc_v2_Y,MIS_Status_v2_default,state_top10
0,84,4,0,0,60000.0,60000.0,48000.0,45,3.5,4.4472,0.67,2.3377,-0.59,0,1,0,0,1,1,0,0,1,0,0
1,60,2,0,0,40000.0,40000.0,32000.0,72,3.5,4.4472,0.67,2.3377,-0.59,0,1,0,0,1,1,0,0,1,0,0
2,180,7,0,0,287000.0,287000.0,215250.0,62,3.5,4.4472,0.67,2.3377,-0.59,1,0,0,0,1,1,0,1,0,0,0
3,60,2,0,0,35000.0,35000.0,28000.0,0,4.1,4.4472,0.67,2.3377,-0.59,1,0,0,0,1,1,0,0,1,0,0
4,240,14,7,7,229000.0,229000.0,229000.0,0,4.8,4.4472,0.67,2.3377,-0.59,1,0,0,0,1,1,0,1,0,0,1


In [4]:
target = 'MIS_Status_v2_default'
features = sba_loans.drop(columns='MIS_Status_v2_default', axis=1).columns

## 3. XGBoosting Classifier with Standard Scalar

In [5]:
X_train_ss, X_test_ss, y_train_ss, y_test_ss = processing_pipeline(sba_loans, target)

### A. Simple  XGBoosting Model

In [ ]:
xgb_ss_mod1 = XGBClassifier(random_state=42, verbosity=0)
y_pred = createModel(xgb_ss_mod1, X_train_ss, y_train_ss, X_test_ss)

### B. Evaluation Metrics

In [ ]:
acc_score, cr, f1, mcc = createClassificationMetrics(y_pred, y_test_ss)

**Accuracy Score**

In [ ]:
print(acc_score)

**Classification Report**

In [ ]:
print(cr)

**F1 Score**

In [ ]:
print(f1)

**Matthew's Correlation Coefficient**

In [ ]:
print(mcc)

### C. Finding Best  XGBoosting Model with Standard Scaler

In [ ]:
param_grid = {
    'learning_rate': [0.1, 1],
    'n_estimators': [50, 100,
    'max_depth': [3, 10]
}

In [ ]:
xgbc = XGBClassifier(random_state=42, verbosity=0)
xgbc_ss_best_params, y_pred = runGridSearchAnalysis(xgbc, param_grid, X_train_ss, y_train_ss, X_test_ss)

In [ ]:
print(xgbc_ss_best_params)

In [ ]:
acc_score, cr, f1, mcc = createClassificationMetrics(y_pred, y_test_ss)
print('Accuracy Score: {}'.format(acc_score))
print('Matthew\'s Correlation Coefficient: {}'.format(mcc))
print('F1 Score: {}'.format(f1))
print('Classification Report: \n{}'.format(cr))

### D. Confusion Matrix

In [ ]:
xgbc_ss_fp = createConfusionMatrix(y_test_ss, y_pred)

### E. Feature Importance with Best Params

In [ ]:
createFeatureImportanceChart(XGBClassifier, xgbc_ss_best_params, features, X_train_ss, y_train_ss)

## 4. XGBoosting Classification with Robust Scaler

In [ ]:
X_train_rs, X_test_rs, y_train_rs, y_test_rs = processing_pipeline(sba_loans, target, scaler='Robust')

### A. Simple Gradient Boosting Model

In [ ]:
xgbc_rs_mod1 = XGBClassifier(random_state=42, verbosity=0)
y_pred = createModel(xgbc_rs_mod1, X_train_rs, y_train_rs, X_test_rs)

### B. Evaluation Metrics

In [ ]:
acc_score, cr, f1, mcc = createClassificationMetrics(y_pred, y_test_rs)

**Accuracy Score**

In [ ]:
print(acc_score)

**Classification Report**

In [ ]:
print(cr)

**F1 Score**

In [ ]:
print(f1)

**Matthew's Correlation Coefficient**

In [ ]:
print(mcc)

### C. Finding Best Gradient Boosting Model with Robust Scaler

In [ ]:
param_grid = {
    'learning_rate': [0.1, 1],
    'n_estimators': [50, 100,
    'max_depth': [3, 10]
}

In [ ]:
xgbc = XGBClassifier(random_state=42)
xgbc_rs_best_params, y_pred = runGridSearchAnalysis(xgbc, param_grid, X_train_rs, y_train_rs, X_test_rs)

In [ ]:
print(xgbc_rs_best_params)

In [ ]:
acc_score, cr, f1, mcc = createClassificationMetrics(y_pred, y_test_rs)
print('Accuracy Score: {}'.format(acc_score))
print('Matthew\'s Correlation Coefficient: {}'.format(mcc))
print('F1 Score: {}'.format(f1))
print('Classification Report: \n{}'.format(cr))

### D. Confusion Matrix

In [ ]:
xgbc_rs_fp = createConfusionMatrix(y_test_rs, y_pred)

### E. Feature Importance with Best Params

In [ ]:
createFeatureImportanceChart(XGBClassifier, xgbc_rs_best_params, features, X_train_rs, y_train_rs)

## 5. Best Gradient Boosting Model

In [ ]:
if xgbc_ss_fp > xgbc_rs_fp:
    print('Best Logistic Regression Model Params: {}'.format(xgbc_rs_best_params))
    print('Best Scaler: Robust Scaler')
else:
    print('Best Logistic Regression Model Params: {}'.format(xgbc_ss_best_params))
    print('Best Scaler: Standard Scaler')